Este archivo genera un split del dataset de imagenes y sus correspondientes asignaciones en csv. Genera un subconjunto de entrenamiento con 100 imagénes por cada clase, y uno de test con 10 imágenes por cada clase. <br>

Primero importamos las librerías necesarias.

In [31]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import shutil

Creamos las constantes que utilizaremos

In [32]:
RANDOM_STATE = 42
CWD = os.getcwd()
DATA_PATH = os.path.join(CWD, "dataset")

A continuación, se implementa el método que se encargará de dividir el dataset original en un subconjunto con 100 imagenes de entrenamiento y 10 de test por clase. Son 9 clases, pero tan se generan 800 imagenes porque la clase 'UNK' no ha sido detectada en ninguna imagen. Además de dividir el conjunto de imagenes ($X$), crea el subconjunto ($y$) de los csv que etiquetan cada imagen. 

In [ ]:
def split_train_test(): 
    try:
        os.mkdir(path=os.path.join(DATA_PATH, "train"))
        os.mkdir(path=os.path.join(DATA_PATH, "test"))
        os.mkdir(path=os.path.join(DATA_PATH, "train/images"))
        os.mkdir(path=os.path.join(DATA_PATH, "test/images"))
    except FileExistsError:
        pass

    df_ISIC = pd.read_csv("dataset/ISIC_2019_Training_GroundTruth.csv")
    df_ISIC = df_ISIC.drop('UNK', axis = 1)

    y_train = pd.DataFrame()
    y_test = pd.DataFrame()

    clases = df_ISIC.columns[1:]
    
    for clase in clases:
        clase_110 = df_ISIC[df_ISIC[clase] == 1].sample(110, random_state=RANDOM_STATE)
        y_train = pd.concat([y_train, clase_110.head(100)])
        y_test = pd.concat([y_test, clase_110.tail(10)])

    y_train["class"] = np.where(y_train["MEL"] == 1, 0, 
                                np.where(y_train["NV"] == 1, 1,
                                np.where(y_train["BCC"] == 1, 2,
                                np.where(y_train["AK"] == 1, 3,
                                np.where(y_train["BKL"] == 1, 4,
                                np.where(y_train["DF"] == 1, 5,
                                np.where(y_train["VASC"] == 1, 6, 
                                np.where(y_train["SCC"] == 1, 7,
                                        -1
                                ))))))))
    
    
    y_test["class"] = np.where(y_test["MEL"] == 1, 0, 
                                np.where(y_test["NV"] == 1, 1,
                                np.where(y_test["BCC"] == 1, 2,
                                np.where(y_test["AK"] == 1, 3,
                                np.where(y_test["BKL"] == 1, 4,
                                np.where(y_test["DF"] == 1, 5,
                                np.where(y_test["VASC"] == 1, 6, 
                                np.where(y_test["SCC"] == 1, 7,
                                        -1
                                ))))))))
    
    y_train = y_train.drop(columns=clases)
    y_test = y_test.drop(columns=clases)

    # Generar csv de train y de test
    print("Generating csv files...")
    y_train.to_csv("dataset/train/y_train.csv", index=False)
    y_test.to_csv("dataset/test/y_test.csv", index=False)

    print("Splitting train, test images in their respective folders...")
    for image_id in y_train["image"]:
        shutil.copy(f"dataset/images/{image_id}.jpg", f"dataset/train/images/{image_id}.jpg")
    
    for image_id in y_test["image"]:
        shutil.copy(f"dataset/images/{image_id}.jpg", f"dataset/test/images/{image_id}.jpg")

    return y_train, y_test

y_train, y_test = split_train_test()



Generating csv files...
Splitting train, test images in their respective folders...
